In [1]:
import os

import numpy.random

os.chdir("C:\\Users\\86173\\Desktop\\eutopia\\my-ML-lib")

In [2]:
import torch

In [3]:
import numpy as np

In [4]:
from torchvision import datasets, transforms

In [5]:
from torch.utils.data import DataLoader

In [6]:
from MyDataLoader import *
from MyCriterion import *
from MyOptimizer import *
from MyLayer import *
from MyNeuralNetworks import *

In [7]:
numpy.random.seed(42)
torch.manual_seed(42)

In [8]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为张量
    # transforms.Normalize((0.5,), (0.5,))  # 归一化
    transforms.Lambda(lambda x: x.view(-1)),  # 张量转换为一维
])
target_transform = transforms.Compose([
    transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1)),  # onehot
])

# 加载训练和测试数据集
# m * (torch.Size([1, 28, 28]), int), 每个样本为(X, y)元组, 其中X是图片, y是标签
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform,
    target_transform=target_transform
    )
test_dataset = datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform,
    target_transform=target_transform
    )

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [9]:
# 定义一个简单的神经网络
class SimpleNN(torch.nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 定义层：一个两层的全连接网络
        self.fc1 = torch.nn.Linear(input_size, inner_size)  # 输入层到隐藏层
        self.fc2 = torch.nn.Linear(inner_size, output_size)  # 隐藏层到输出层

    def forward(self, x):
        x = x.view(-1, input_size)  # 展平输入（假设输入是28x28图像）
        x = torch.relu(self.fc1(x))  # 激活函数 ReLU
        x = self.fc2(x)  # 输出层
        return x


In [10]:
class MyNN(MyNeuralNetwork):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyNN, self).__init__()
        fc1 = FC_Layer(input_size, hidden_size)
        relu1 = ReLu_Layer(hidden_size, hidden_size)
        fc2 = FC_Layer(hidden_size, output_size)
        self.layers = [fc1, relu1, fc2]
        self.update_layer_list = [fc1, fc2]

In [11]:
import torch.optim as optim
# 训练函数
def train(model, train_loader, criterion, optimizer, num_epochs=5, lr=1e-3, my_model=True):
    for epoch in range(num_epochs):
        for images, labels in train_loader:
            # 前向传播
            if my_model:
                images = images.numpy()
                labels = labels.numpy()
                
                outputs = model.forward(images)
                loss = criterion.forward(outputs, labels)
                
                in_grad = criterion.backward()
                model.backward(in_grad)
                grads = model.get_grads()
                optimizer.step(grads)  # 更新参数
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                # 反向传播和优化
                optimizer.zero_grad()  # 清除上一步的梯度
                loss.backward()  # 计算梯度
                optimizer.step()  # 更新参数

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [12]:
def evaluate(model, test_loader, my_model=True):
    if not my_model:
        model.eval()  # 切换到评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            if my_model:
                images = images.numpy()
                outputs = model.forward(images)
                outputs = torch.from_numpy(outputs)
            else:
                outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test dataset: {100 * correct / total:.2f}%')


In [13]:
input_size = 28 * 28
inner_size = 128
output_size = 10

In [ ]:
# 创建网络实例
my_model = MyNN(input_size, inner_size, output_size)
params = my_model.get_params()
optimizer = SimpleSGD(params)
criterion = CrossEntropyLoss() 

print("MyNN", f"隐藏层大小:{inner_size}", "SimpleSGD优化器")
train(my_model, train_loader, criterion, optimizer, my_model=True)
evaluate(my_model, test_loader, my_model=True)

In [ ]:
# 创建网络实例
my_model = MyNN(input_size, inner_size, output_size)
params = my_model.get_params()
optimizer = Adam(params)
criterion = CrossEntropyLoss() 

print("MyNN", f"隐藏层大小:{inner_size}", "Adam优化器")
train(my_model, train_loader, criterion, optimizer, my_model=True)
evaluate(my_model, test_loader, my_model=True)

In [ ]:
inner_size = 256

In [ ]:
my_model = MyNN(input_size, inner_size, output_size)
params = my_model.get_params()
optimizer = SimpleSGD(params)

print("MyNN", f"隐藏层大小:{inner_size}", "SimpleSGD优化器")
criterion = CrossEntropyLoss() 
train(my_model, train_loader, criterion, optimizer, my_model=True)
evaluate(my_model, test_loader, my_model=True)

In [ ]:
my_model = MyNN(input_size, inner_size, output_size)
params = my_model.get_params()
optimizer = Adam(params)
criterion = CrossEntropyLoss() 

print("MyNN", f"隐藏层大小:{inner_size}", "Adam优化器")
train(my_model, train_loader, criterion, optimizer, my_model=True)
evaluate(my_model, test_loader, my_model=True)

In [ ]:
inner_size = 128

In [ ]:
# 创建网络实例
torch_model = SimpleNN()
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)  # 使用 Adam 优化器# 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()  # 交叉熵损失（用于分类任务）

print("TorchNN", f"隐藏层大小:{inner_size}", "Adam优化器")
# 开始训练
train(torch_model, train_loader, criterion, optimizer, my_model=False)

# 评估模型
evaluate(torch_model, test_loader, my_model=False)

In [14]:
inner_size = 256

In [ ]:
# 创建网络实例
torch_model = SimpleNN()
optimizer = optim.Adam(torch_model.parameters(), lr=0.001)  # 使用 Adam 优化器# 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()  # 交叉熵损失（用于分类任务）

print("TorchNN", f"隐藏层大小:{inner_size}", "Adam优化器")
# 开始训练
train(torch_model, train_loader, criterion, optimizer, my_model=False)

# 评估模型
evaluate(torch_model, test_loader, my_model=False)

TorchNN 隐藏层大小:256 Adam优化器
